In [24]:
import time
import pandas as pd
import numpy as np  
import firebase_admin
from firebase_admin import credentials, db

In [25]:
cred = credentials.Certificate("creds.json")
firebase_admin.initialize_app(cred, {
    "databaseURL": "https://alert-8c0c7-default-rtdb.asia-southeast1.firebasedatabase.app/"
})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [26]:
accident_locations = db.reference("/accident_locations")
df = pd.read_csv("complete_data.csv")  # Hospital dataset

In [27]:
def closest_hospital(lat, long):
    grid = 0.01
    find = False
    n = 1

    while not find:
        start_lat = lat - grid * n
        end_lat = lat + grid * n
        start_long = long - grid * n
        end_long = long + grid * n

        test = df[
            (df['latitude'] >= start_lat) & (df['latitude'] <= end_lat) &
            (df['longitude'] >= start_long) & (df['longitude'] <= end_long)
        ]

        if not test.empty:
            find = True
        else:
            n += 1

    distances = np.square(test['latitude'] - lat) + np.square(test['longitude'] - long)
    min_index = distances.idxmin()

    return df.loc[min_index, 'hospital Name']

In [28]:
def update_nearest_hospitals():
    while True:
        accident_data = accident_locations.get()
        if not accident_data:
            time.sleep(5)
            continue  # Skip iteration if no accident data

        for id, value in accident_data.items():
            if value['Nearest_hospital'] == "":
                lat, lon = value['latitude'], value['longitude']
                hospital = closest_hospital(lat, lon)
                accident_locations.update({f'/{id}/Nearest_hospital': hospital})

        time.sleep(5)

In [29]:
update_nearest_hospitals()

KeyboardInterrupt: 